In [37]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas import Timestamp
import openpyxl
from openpyxl.styles import PatternFill

# Function

def generate_monthly_timestamps(start_date, end_date):
    current_date = start_date.replace(day=1)

    monthly_timestamps = []

    while current_date <= end_date:
        end_of_month = current_date + relativedelta(day=31)
        monthly_timestamps.append(end_of_month)
        current_date += relativedelta(months=1)

    return monthly_timestamps

# Mengakses file Excel dengan path baru
file_path = 'Book6.xlsx'

# Membaca file Excel
excel_data = pd.read_excel(file_path, sheet_name=None)

# Menjadikan sheet QC sebagai database
qc_data = excel_data['QC']

# Menulis ulang pada sheet OUTPUT
output_data = qc_data.copy()  # Meng-copy data dari sheet QC ke OUTPUT

#Menyimpan perubahan pada sheet OUTPUT
excel_data['OUTPUT'] = output_data

#Menampilkan data pada sheet TANDA
tanda_data = excel_data['TANDA']

output_data['Date'] = pd.to_datetime(output_data['Date'], errors='coerce')

uwi_and_time_output = []
for i in output_data.index:
    if [output_data['UWI'][i], output_data['Date'][i]] in uwi_and_time_output:
        continue
    else:
        timestamp = output_data['Date'][i]
        uwi_and_time_output.append(
            [output_data['UWI'][i], timestamp])
        

        
        
results = []
hasil = []
sand_final = []
# Iterasi untuk mengisi nilai "P" pada kolom PERFO SAND di sheet TANDA
for index, row in uwi_and_time_output:
    uwi = index
    date = row

    # Temukan nilai PERFO SAND yang sesuai dengan UWI dan Date dari sheet OUTPUT_UNIQUE
    rs = []
    for i in output_data.index:
        if [output_data['UWI'][i] , output_data['Date'][i]] == [uwi, date]:
            if output_data['Perf Status'][i] == 'perforation':
                rs.append(([output_data['Perf Base (ftMD)'][i] - output_data['Perf Top (ftMD)'][i],
                    output_data['PERFO SAND'][i], output_data['Perf Status'][i], date, uwi]))
            elif output_data['Perf Status'][i] == 'squeeze':
                rs.append(([output_data['Perf Top (ftMD)'][i] - output_data['Perf Base (ftMD)'][i],
                            output_data['PERFO SAND'][i], output_data['Perf Status'][i], date, uwi]))
    results.append(rs)
    
    sand = []
    for i in rs:
        if i[1] in sand:
            continue
        else:
            sand.append(i[1])
    sand_final.append(sand)
    

for i in range(len(sand_final)):
    selisih_sand = []
    for j in sand_final[i]:
            selisih = []
            date_sand = []
            uwi_sand = []
            for k in results[i]:
                if j == k[1]:
                    selisih.append(k[0])
            selisih = sum(selisih)
            date_sand = results[i][0][3]
            uwi_sand = results[i][0][4]
            selisih_sand.append([j, selisih, date_sand, uwi_sand])
    hasil.append(selisih_sand)
    
    
hasil_akhir = []
count = 0
for i in range(len(hasil)):
    count += 1
    for j in hasil[i]:
        found = False
        if len(hasil_akhir) != 0:
            if j[3] != hasil_akhir[0][0][3]:
                keyword = hasil_akhir[0][0][3]
                filtered_data = [
                    inner_list for inner_list in hasil_akhir if inner_list[0][-1] != keyword]
                hasil_akhir = filtered_data
            else:
                for k, sublist in enumerate(hasil_akhir):
                    if j[0] == sublist[0][0] and j[3] == sublist[0][3]:
                        hasil_akhir[k][0][1] += j[1]
                        found = True
                        for l in hasil_akhir:
                            if l[0][1] <= 0:
                                l[1] = "squeeze"
                                l[0][3] = j[3]
                                l[0][2] = j[2]
                            else:
                                l[1] = "perforation"
                                l[0][3] = j[3]
                                l[0][2] = j[2]
                        break
        if not found and j[1] > 0:
            hasil_akhir.append([j,"perforation"])
        elif not found and j[1] <= 0:
            hasil_akhir.append([j,"squeeze"])
    
    if i == len(hasil)-1:
        print(hasil_akhir)
        continue
    else:
        if uwi_and_time_output[i][0] != uwi_and_time_output[i+1][0]:
            print(hasil_akhir)
            continue
        else:
            start_date = uwi_and_time_output[i][1]
            end_date = uwi_and_time_output[i+1][1]
            delta_time = generate_monthly_timestamps(
            start_date, end_date)
            
    print(hasil_akhir)
    #copy hasil akhir to every delta time
    # for i in range(len(delta_time)):
    #     hasil_akhir.append([hasil_akhir[0][0], hasil_akhir[0][1]])
    
    

# # Menyimpan perubahan pada sheet TANDA
# excel_data['TANDA'] = tanda_data

# # Menyimpan ke file Excel
# with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
#     for sheet, data in excel_data.items():
#         data.to_excel(writer, sheet_name=sheet, index=False)

[[['T_BKA', 1.0, Timestamp('1971-07-12 00:00:00'), 'PETP00001V1'], 'perforation'], [['T_BKB', 8.0, Timestamp('1971-07-12 00:00:00'), 'PETP00001V1'], 'perforation'], [['T_BKC1', 126.0, Timestamp('1971-07-12 00:00:00'), 'PETP00001V1'], 'perforation'], [['T_BKC2', -37.0, Timestamp('1971-07-12 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKC3', 24.0, Timestamp('1971-07-12 00:00:00'), 'PETP00001V1'], 'perforation']]
[[['T_BKA', -28.0, Timestamp('1975-04-13 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKB', -35.0, Timestamp('1975-04-13 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKC1', 94.0, Timestamp('1975-04-13 00:00:00'), 'PETP00001V1'], 'perforation'], [['T_BKC2', -37.0, Timestamp('1975-04-13 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKC3', 24.0, Timestamp('1975-04-13 00:00:00'), 'PETP00001V1'], 'perforation']]
[[['T_BKA', -36.0, Timestamp('1975-08-25 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKB', -35.0, Timestamp('1975-08-25 00:00:00'), 'PETP00001V1'], 'squeeze'], [['T_BKC1', 12